### Package

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch.autograd import Variable
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

torch.manual_seed(1)    # reproducible

### Pytorch Class

In [2]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)   # hidden layer
        self.out = torch.nn.Linear(n_hidden, n_output)   # output layer

    def forward(self, x):
        x = F.relu(self.hidden(x))      # activation function for hidden layer
        x = self.out(x)
        return x

### Data load

In [3]:
dataset = pd.read_csv('CleanedData _pytorch.csv')
dataset = np.array(dataset)

data = dataset[:, :-1]
target = dataset[:, -1]

In [4]:
random_seed = 43
x_train, x_test, y_train, y_test = train_test_split(data, target, test_size=0.33,random_state=random_seed)

x_train_tensor = torch.tensor(x_train).type(torch.FloatTensor)
x_test_tensor = torch.tensor(x_test).type(torch.FloatTensor)
y_train_tensor = torch.tensor(y_train).type(torch.LongTensor)
y_test_tensor = torch.tensor(y_test).type(torch.LongTensor)

x_train_tensor, x_test_tensor, y_train_tensor, y_test_tensor = Variable(x_train_tensor), Variable(x_test_tensor), Variable(y_train_tensor), Variable(y_test_tensor)

In [9]:
net = Net(n_feature=42, n_hidden=128, n_output=3)     # define the network
print(net)  # net architecture

# Loss and Optimizer
# Softmax is internally computed.
# Set parameters to be updated.
optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
loss_func = torch.nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

Net(
  (hidden): Linear(in_features=42, out_features=128, bias=True)
  (out): Linear(in_features=128, out_features=3, bias=True)
)


In [10]:
for t in range(10000):
    out = net(x_train_tensor)                 # input x and predict based on x
    loss = loss_func(out, y_train_tensor)     # must be (1. nn output, 2. target), the target label is NOT one-hotted

    optimizer.zero_grad()   # clear gradients for next train
    loss.backward()         # backpropagation, compute gradients
    optimizer.step()        # apply gradients

    if t % 1000 == 0:
        _, prediction = torch.max(F.softmax(out, dim=1), 1)
        pred_x_train = prediction.data.numpy().squeeze()
        target_x_train = y_train_tensor.data.numpy()
        accuracy = np.mean(pred_x_train == target_x_train)*100
        print(accuracy)


30.591458429994915
69.65378582002165
74.83263736992113
76.8608735997879
77.78219659309339
78.46932237467135
79.22714920129913
79.75298822385717
80.13742515631559
80.4887209739069


In [11]:
_, prediction_test = torch.max(F.softmax(net(x_test_tensor),dim=1), 1)
pred_x_test = prediction_test.data.numpy().squeeze()
target_x_test = y_test_tensor.data.numpy()
np.mean(pred_x_test == target_x_test)*100

79.20965282138692